In [0]:
%sql

CREATE SCHEMA IF NOT EXISTS hive_metastore.bordereaux

In [0]:
# %sql

# DROP SCHEMA IF EXISTS hive_metastore.bordereaux CASCADE

In [0]:
%sql
SELECT a.policy_number, a.endorsement_number, a.commit_date, a.coverage_premium, a.terrorism_premium, 
         COALESCE(a.coverage_premium, 0) + COALESCE(a.terrorism_premium, 0) AS gross_premium
  FROM hive_metastore.bordereaux.agricultural a WHERE a.policy_number = 'NCF-1009564-00' AND a.endorsement_number = '4'

In [0]:
%sql
WITH policy_list AS (
  SELECT 'NBA-1000305-01' AS policy_number, 1 AS endorsement_number
),
params AS (
  SELECT
    DATE '2020-01-01' AS start_date,
    DATE '2020-12-31' AS end_date
),
combined AS (
  SELECT 'agricultural' AS source_table, a.policy_number, a.endorsement_number, a.commit_date,
         COALESCE(a.coverage_premium, 0) + COALESCE(a.terrorism_premium, 0) AS gross_premium
  FROM hive_metastore.bordereaux.agricultural a
  JOIN policy_list pl ON a.policy_number = pl.policy_number
                     AND a.endorsement_number = pl.endorsement_number
  JOIN params p ON CAST(a.commit_date AS DATE) >= p.start_date
               AND CAST(a.commit_date AS DATE) < p.end_date
 
  UNION ALL
 
  SELECT 'auto_commercial', ac.policy_number, ac.endorsement_number, ac.commit_date,
         ac.prorated_prem AS gross_premium
  FROM hive_metastore.bordereaux.commercial_auto ac
  JOIN policy_list pl ON ac.policy_number = pl.policy_number
                     AND ac.endorsement_number = pl.endorsement_number
  JOIN params p ON CAST(ac.commit_date AS DATE) >= p.start_date
               AND CAST(ac.commit_date AS DATE) < p.end_date
 
  UNION ALL
 
  SELECT 'inland_marine', im.policy_number, im.endorsement_number, im.commit_date,
         im.prorate_premium AS gross_premium
  FROM hive_metastore.bordereaux.inland_marine im
  JOIN policy_list pl ON im.policy_number = pl.policy_number
                     AND im.endorsement_number = pl.endorsement_number
  JOIN params p ON CAST(im.commit_date AS DATE) >= p.start_date
               AND CAST(im.commit_date AS DATE) < p.end_date
 
  UNION ALL
 
  SELECT 'property', pr.policy_number, pr.endorsement_number, pr.commit_date,
         pr.coverage_premium AS gross_premium
  FROM hive_metastore.bordereaux.property pr
  JOIN policy_list pl ON pr.policy_number = pl.policy_number
                     AND pr.endorsement_number = pl.endorsement_number
  JOIN params p ON CAST(pr.commit_date AS DATE) >= p.start_date
               AND CAST(pr.commit_date AS DATE) < p.end_date
 
  UNION ALL
 
  SELECT 'crime', c.policy_number, c.endorsement_number, c.commit_date,
         c.pro_rate AS gross_premium
  FROM hive_metastore.bordereaux.crime c
  JOIN policy_list pl ON c.policy_number = pl.policy_number
                     AND c.endorsement_number = pl.endorsement_number
  JOIN params p ON CAST(c.commit_date AS DATE) >= p.start_date
               AND CAST(c.commit_date AS DATE) < p.end_date
 
  UNION ALL
 
  SELECT 'liability', l.policy_number, l.endorsement_number, l.commit_date,
         l.premium AS gross_premium
  FROM hive_metastore.bordereaux.liability l
  JOIN policy_list pl ON l.policy_number = pl.policy_number
                     AND l.endorsement_number = pl.endorsement_number
  JOIN params p ON CAST(l.commit_date AS DATE) >= p.start_date
               AND CAST(l.commit_date AS DATE) < p.end_date
)
 
SELECT source_table, policy_number, endorsement_number, commit_date,
       SUM(gross_premium) AS total_gross_premium
FROM combined
GROUP BY source_table, policy_number, endorsement_number, commit_date
ORDER BY source_table, commit_date;

In [0]:
df = spark.sql("SELECT policy_number, commit_date, 'auto_commercial' FROM hive_metastore.bordereaux.auto_commercial LIMIT 2;")
df.display()

df = spark.sql("SELECT policy_number, commit_date, 'crime' FROM hive_metastore.bordereaux.crime LIMIT 2;")
df.display()

df = spark.sql("SELECT policy_number, commit_date, 'fee' FROM hive_metastore.bordereaux.fee LIMIT 2;")
df.display()

df = spark.sql("SELECT policy_number, commit_date, 'inland_marine' FROM hive_metastore.bordereaux.inland_marine LIMIT 2;")
df.display()

In [0]:
df = spark.sql("SELECT MAX(YEAR(policy_effective_date)) AS year FROM hive_metastore.bordereaux.agricultural_cleaned;")
df.display()

df = spark.sql("SELECT policy_effective_date FROM hive_metastore.bordereaux.agricultural_cleaned ORDER BY policy_effective_date DESC;")
df.display()

In [0]:
# from pyspark.sql import SparkSession

# spark.sql("CREATE SCHEMA IF NOT EXISTS hive_metastore.bordereaux;")

In [0]:
# files = dbutils.fs.ls("/mnt/bordereaux/bronze/2020")
# file_name = "agricultural-bordereaux-2020.csv"
# any(file_info.name == file_name for file_info in files) 

In [0]:
df = spark.sql(f"SELECT MAX(YEAR(policy_effective_date)) AS year FROM hive_metastore.bordereaux.commercial_auto;")
display(df)
df = spark.sql("SELECT DISTINCT YEAR(policy_effective_date) AS year_distinct FROM hive_metastore.bordereaux.commercial_auto ORDER BY year_distinct DESC;")
display(df)

# # df = spark.sql(f"SELECT DISTINCT policy_effective_date FROM hive_metastore.bordereaux.commercial_auto ORDER BY policy_effective_date;")
# # display(df)

# df = spark.sql(f"SELECT DISTINCT CAST(policy_effective_date AS DATE) FROM hive_metastore.bordereaux.commercial_auto ORDER BY policy_effective_date;")
# display(df)

In [0]:
df = spark.sql(f"SELECT DISTINCT policy_effective_date AS year_distinct FROM hive_metastore.bordereaux.commercial_auto ORDER BY year_distinct DESC;")
display(df)

In [0]:
df = spark.sql(f"SELECT * FROM hive_metastore.bordereaux.commercial_auto WHERE policy_effective_date = 'Policy Effective Date';")
display(df)

In [0]:
df = spark.sql(f"SELECT * FROM hive_metastore.bordereaux.commercial_auto WHERE policy_effective_date = '44957';")
display(df)

In [0]:
spark.sql("""
    DELETE FROM hive_metastore.bordereaux.commercial_auto
    WHERE policy_effective_date = 'Policy Effective Date'
""")

In [0]:
for table in [
    "agricultural", "commercial_auto", "crime", "cyber", "epli", "equipment_breakdown",
    "fee", "inland_marine", "liability", "property", "umbrella"
]:
    df = spark.sql(f"SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS year, '{table}' AS table_name FROM hive_metastore.bordereaux.{table};")
    display(df)
    # df = spark.sql(f"SELECT DISTINCT YEAR(policy_effective_date) AS year_distinct, '{table}' AS table_name FROM hive_metastore.bordereaux.{table} ORDER BY year_distinct DESC;")
    # display(df)

In [0]:
%sql

SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'agricultural' AS table_name FROM hive_metastore.bordereaux.agricultural
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'commercial_auto' AS table_name FROM hive_metastore.bordereaux.commercial_auto
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'crime' AS table_name FROM hive_metastore.bordereaux.crime
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'cyber' AS table_name FROM hive_metastore.bordereaux.cyber
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'epli' AS table_name FROM hive_metastore.bordereaux.epli
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'equipment_breakdown' AS table_name FROM hive_metastore.bordereaux.equipment_breakdown
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'fee' AS table_name FROM hive_metastore.bordereaux.fee
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'inland_marine' AS table_name FROM hive_metastore.bordereaux.inland_marine
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'liability' AS table_name FROM hive_metastore.bordereaux.liability
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'property' AS table_name FROM hive_metastore.bordereaux.property
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'umbrella' AS table_name FROM hive_metastore.bordereaux.umbrella

In [0]:
%sql

SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'agricultural' AS table_name FROM hive_metastore.bordereaux.agricultural
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'commercial_auto' AS table_name FROM hive_metastore.bordereaux.commercial_auto
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'crime' AS table_name FROM hive_metastore.bordereaux.crime
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'cyber' AS table_name FROM hive_metastore.bordereaux.cyber
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'epli' AS table_name FROM hive_metastore.bordereaux.epli
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'equipment_breakdown' AS table_name FROM hive_metastore.bordereaux.equipment_breakdown
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'fee' AS table_name FROM hive_metastore.bordereaux.fee
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'inland_marine' AS table_name FROM hive_metastore.bordereaux.inland_marine
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'liability' AS table_name FROM hive_metastore.bordereaux.liability
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'property' AS table_name FROM hive_metastore.bordereaux.property
UNION
SELECT COUNT(*) AS total_row, MAX(YEAR(policy_effective_date)) AS max_year, 'corp_dev' AS envi, 'umbrella' AS table_name FROM hive_metastore.bordereaux.umbrella

In [0]:
# In a Databricks notebook cell
schema_name = "bordereaux"  # Replace with your schema name

# Get all table names in the schema
tables_df = spark.sql(f"SHOW TABLES IN {schema_name}")
table_names = [row.tableName for row in tables_df.collect()]

# Get details for each table
table_sizes = []
for table_name in table_names:
    try:
        detail_df = spark.sql(f"DESCRIBE DETAIL {schema_name}.{table_name}")
        detail = detail_df.collect()[0]
        table_sizes.append({
            'schema_name': schema_name,
            'table_name': table_name,
            'size_in_bytes': detail.sizeInBytes,
            'size_in_mb': round(detail.sizeInBytes / (1024 * 1024), 2),
            'size_in_gb': round(detail.sizeInBytes / (1024 * 1024 * 1024), 2),
            'num_files': detail.numFiles,
            'location': detail.location
        })
    except Exception as e:
        print(f"Error getting details for {table_name}: {e}")

# Convert to DataFrame and display
sizes_df = spark.createDataFrame(table_sizes)
display(sizes_df.orderBy('size_in_bytes', ascending=False))

In [0]:
%sql

USE hive_metastore.bordereaux;

ALTER TABLE umbrella 
ALTER COLUMN policy_effective_date TYPE DATE;